# **Análisis Exploratorio de Datos (EDA)**

Realizaremos un Análisis Exploratorio de Datos completo utilizando un conjunto de datos real. Aplicaremos todos los conceptos que hemos aprendido hasta ahora, incluyendo limpieza, visualización, manejo de datos faltantes y outliers, procesamiento de datos categóricos, y más.


## **Descripción del Problema**

Imagina que eres un científico de datos en una empresa de telecomunicaciones. La empresa está preocupada por el alto índice de abandono de clientes (churn) y te han pedido que analices los datos de clientes para identificar patrones y factores que contribuyen al churn. Tu objetivo es realizar un EDA completo para extraer insights valiosos que puedan ayudar a la empresa a retener clientes.


## **1. Carga de datos**

Revisamos a lo largo del curso, diferentes maneras de acceder a la información. Por ejemplo, con bases SQL, noSQL, APIs, Scrapping, leyendo .csv, etc...
En este caso en particular, accederemos a los datos directamente desde el repo de github en donde están alojados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuración de gráficos
%matplotlib inline
plt.style.use('ggplot')
sns.set(rc={'figure.figsize':(10,6)})

In [ ]:
# Descargar stopwords si es necesario
nltk.download('stopwords')

# Cargar el conjunto de datos
url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url)

In [ ]:
df.head(5)

### Descripción de las columnas:

* customerID: Identificador único de cada cliente. Es una cadena de caracteres que distingue a cada cliente en el conjunto de datos.

* gender: Género del cliente. Los valores posibles son:

  Male: Hombre.

  Female: Mujer.

* SeniorCitizen: Indica si el cliente es una persona de la tercera edad. Los valores son:

    1: El cliente es un adulto mayor (normalmente se considera a partir de 65 años).

    0: El cliente no es un adulto mayor.

* Partner: Indica si el cliente tiene pareja. Los valores son:

    Yes: El cliente tiene pareja.

    No: El cliente no tiene pareja.

* Dependents: Indica si el cliente tiene personas dependientes a su cargo (como hijos o familiares). Los valores son:

    Yes: El cliente tiene dependientes.

    No: El cliente no tiene dependientes.

* tenure: Número de meses que el cliente ha estado con la empresa. Esta columna representa la antigüedad del cliente en términos de meses. Por ejemplo:

* PhoneService: Indica si el cliente tiene servicio telefónico. Los valores son:

    Yes: El cliente tiene servicio telefónico.

    No: El cliente no tiene servicio telefónico.

* MultipleLines: Indica si el cliente tiene múltiples líneas telefónicas. Los valores son:

    Yes: El cliente tiene múltiples líneas.

    No: El cliente no tiene múltiples líneas.

* No phone service: El cliente no tiene servicio telefónico.

* InternetService: Tipo de servicio de internet del cliente. Los valores son:

* DSL: El cliente tiene servicio de internet DSL.

* Fiber optic: El cliente tiene servicio de internet por fibra óptica.

* No: El cliente no tiene servicio de internet.

* OnlineSecurity: Indica si el cliente tiene servicio de seguridad en línea. Los valores son:

    Yes: El cliente tiene seguridad en línea.

  No: El cliente no tiene seguridad en línea.

* No internet service: El cliente no tiene servicio de internet.

* OnlineBackup: Indica si el cliente tiene servicio de respaldo en línea. Valores similares a OnlineSecurity.

* DeviceProtection: Indica si el cliente tiene protección para dispositivos. Valores similares.

* TechSupport: Indica si el cliente tiene soporte técnico. Valores similares.

* StreamingTV: Indica si el cliente tiene servicio de streaming de TV. Valores similares.

* StreamingMovies: Indica si el cliente tiene servicio de streaming de películas. Valores similares.

* Contract: Tipo de contrato del cliente. Los valores son:

    Month-to-month: Contrato mes a mes.

    One year: Contrato de un año.

    Two year: Contrato de dos años.

* PaperlessBilling: Indica si el cliente tiene facturación electrónica. Los valores son:

    Yes: El cliente utiliza facturación electrónica.

    No: El cliente recibe facturas en papel.

* PaymentMethod: Método de pago preferido por el cliente. Los valores son:

    Electronic check: Cheque electrónico.

    Mailed check: Cheque enviado por correo.
  
* Bank transfer (automatic): Transferencia bancaria automática.

* Credit card (automatic): Tarjeta de crédito automática.

* MonthlyCharges: Cargos mensuales que el cliente paga por los servicios contratados. Es un valor numérico en dólares.

* TotalCharges: Cargos totales acumulados por el cliente durante el tiempo que ha estado con la empresa. También es un valor numérico en dólares.

* Churn: Indica si el cliente ha abandonado el servicio. Los valores son:

    Yes: El cliente ha cancelado el servicio.

    No: El cliente continúa con el servicio.


## **2. Exploración inicial de los datos**

Echamos un vistazo a las primeras entradas del conjunto de datos para familiarizarnos con las variables disponibles.

Las variables incluyen información demográfica, detalles del servicio y si el cliente ha abandonado o no el servicio.

In [ ]:
# Información general del DataFrame
df.info()

Observamos el tipo de datos de cada columna y la cantidad de valores no nulos para identificar posibles datos faltantes.

Notamos que hay 7043 registros y 21 columnas. Algunas columnas son numéricas y otras categóricas.


In [ ]:
# Estadísticas descriptivas
df.describe(include='all')

Obtenemos estadísticas descriptivas para todas las variables, lo que nos ayudará a identificar outliers y entender la distribución de los datos.

Observamos que algunas variables parecen tener datos inconsistentes, como **TotalCharges**.


In [ ]:
df['TotalCharges']

**¿Por qué si parece tener únicamente valores numéricos no lo detecto como columna de flotantes y no nos dió sus estadísticos?**

In [ ]:
df['TotalCharges'].dtype

¿Qué nos dice la línea anterior?

Intentaremos convertir la columna a datos numéricos y revisaremos los errores que no se logren convertir

In [ ]:
# Identificamos valores no numéricos en 'TotalCharges'
df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()][['customerID', 'TotalCharges']]

Hemos encontrado que la columna **TotalCharges** debería ser numérica, pero aparece como objeto debido a valores vacíos o no numéricos.

In [ ]:
# Convertimos 'TotalCharges' a numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

Revisemos los datos que hemos convertido a nulos

In [ ]:
# Verificamos datos faltantes
df.isnull().sum()

## **3. Manejo de datos faltantes**

In [ ]:
# Visualizamos datos faltantes
import missingno as msno
msno.matrix(df)
plt.show()

Visualizamos los datos faltantes utilizando la librería **missingno** para identificar patrones en los faltantes.

Observamos que los valores faltantes en **TotalCharges** parecen ser aleatorios.


In [ ]:
# Porcentaje de datos faltantes por columna
(df.isnull().sum() / len(df)) * 100

**TotalCharges** tiene aproximadamente el 0.15% de datos faltantes.

Una opción sensata puede ser eliminar estos registros y continuar con el resto de los datos. Sin embargo, decidiremos imputar estos valores en lugar de eliminar las filas.


In [ ]:
# Imputación de 'TotalCharges' usando el valor de 'MonthlyCharges' por 'tenure'
df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'])

In [ ]:
¿Por qué elegímos esas columnas multiplicadas para imputarlos?

In [ ]:
df[["tenure", "MonthlyCharges", "TotalCharges"]]

In [ ]:
# Verificamos que no queden datos faltantes
df.isnull().sum()

## **4. Detección y tratamiento de outliers**

In [ ]:
# Revisemos las distribuciones de datos del DataFrame
sns.boxplot(data=df)

¿Qué información de aquí nos puede ser relevante?

In [ ]:
# Boxplot de 'MonthlyCharges'
sns.boxplot(x=df['MonthlyCharges'])
plt.title('Distribución de MonthlyCharges')
plt.show()


In [ ]:
# Boxplot de 'Tenure'
sns.boxplot(x=df['tenure'])
plt.title('Distribución de tenure')
plt.show()


In [ ]:
# Boxplot de 'TotalCharges'
sns.boxplot(x=df['TotalCharges'])
plt.title('Distribución de TotalCharges')
plt.show()


In [ ]:
# Boxplot de 'SeniorCitizen'
sns.boxplot(x=df['SeniorCitizen'])
plt.title('Distribución de SeniorCitizen')
plt.show()

¿Qué hubiera pasado en caso de encontrar outliers para alguna de estas variables?

¿Cómo deberíamos manejarlos?

## **5. Procesamiento de datos categóricos y de texto**

Identificamos las variables categóricas para procesarlas adecuadamente.

In [ ]:
# Revisamos las variables categóricas
categorical_features = df.select_dtypes(include=['object']).columns
categorical_features

In [ ]:
# Eliminamos 'customerID' ya que es un identificador único y no tiene sentido procesarlo como variable categórica
df_cat = df.copy()
df_cat.drop('customerID', axis=1, inplace=True)

Aplicamos **Label Encoding** a variables binarias y **One-Hot Encoding** a variables con múltiples categorías.

Esto nos permitirá utilizar estas variables en análisis numéricos y modelos de machine learning.


In [ ]:
df_cat[['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']]

In [ ]:
# Convertimos las variables categóricas a numéricas usando Label Encoding y One-Hot Encoding

# Variables binarias
binary_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

for feature in binary_features:
    df_cat[feature] = df_cat[feature].map({'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0})


In [ ]:
df_cat[['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']]

In [ ]:
df_cat[['MultipleLines', 'InternetService',]]

In [ ]:
# Variables con más de dos categorías
multi_category_features = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                           'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

df_cat = pd.get_dummies(df_cat, columns=multi_category_features)

In [ ]:
df_cat[['MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',]]

Procesamos el texto en una nueva columna que nombraremos: **PaymentMethod_Text** eliminando stopwords y caracteres especiales, convirtiendo a minúsculas, y limpiando el texto.

Esto es útil si quisiéramos hacer un análisis de texto más profund.


In [ ]:
df_cat.columns

In [ ]:
# Extraemos el método de pago preferido para cada cliente
payment_methods = ['PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)',
                   'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']

df_cat['PaymentMethod_Text'] = df_cat[payment_methods].idxmax(axis=1)

# Limpiamos el texto
df_cat['PaymentMethod_Text'] = df_cat['PaymentMethod_Text'].str.replace('PaymentMethod_', '')
df_cat['PaymentMethod_Text'] = df_cat['PaymentMethod_Text'].str.lower()
df_cat['PaymentMethod_Text']

In [ ]:
# Eliminamos stopwords y caracteres especiales
stop_words = set(stopwords.words('english'))
df_cat['PaymentMethod_Text'] = df_cat['PaymentMethod_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df_cat['PaymentMethod_Text'] = df_cat['PaymentMethod_Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df_cat['PaymentMethod_Text']

## **6. Visualización de datos**

Visualizamos la distribución de la variable **tenure** para entender cuánto tiempo llevan los clientes con la empresa.

Visualizamos la distribución de la variable **MonthlyCharges** para explorar los pagos que se hacen mes a mes a la empresa.

Exploramos visualmente, la distribución de clientes por tipo de internet y contrato.

In [ ]:
# Histograma de 'tenure' (meses de suscripción)
sns.histplot(df['tenure'], bins=30, kde=True)
plt.title('Distribución de Tenure')
plt.show()

Observamos que hay un pico en clientes nuevos y en clientes de largo plazo.

Esto sugiere que la compañía tiene tanto clientes nuevos como viejos y leales.

In [ ]:
# Histograma de 'MonthlyCharges' (Cargos Mensuales)
plt.figure(figsize=(10,6))
sns.histplot(df['MonthlyCharges'], bins=30, kde=True, color='green')
plt.title('Distribución de Cargos Mensuales')
plt.xlabel('Cargos Mensuales ($)')
plt.ylabel('Frecuencia')
plt.show()

Los cargos mensuales tienen una distribución bimodal, indicando que hay dos grupos distintos de clientes en términos de lo que pagan mensualmente.

Esto podría deberse a diferentes paquetes o servicios contratados.

In [ ]:
# Gráfico de barras de 'Contract'
plt.figure(figsize=(8,6))
sns.countplot(x='Contract', data=df, palette='pastel')
plt.title('Número de Clientes por Tipo de Contrato')
plt.xlabel('Tipo de Contrato')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=45)
plt.show()

La mayoría de los clientes tienen contratos mes a mes.

Los contratos de un año y dos años son menos populares.

In [ ]:
# Gráfico de barras de 'InternetService'
plt.figure(figsize=(8,6))
sns.countplot(x='InternetService', data=df, palette='muted')
plt.title('Número de Clientes por Tipo de Servicio de Internet')
plt.xlabel('Tipo de Servicio de Internet')
plt.ylabel('Número de Clientes')
plt.show()


El servicio de fibra óptica es el más utilizado.
    
Un número considerable de clientes no tiene servicio de internet.

**Análisis de Churn**

¿Afectará el tipo de contrato?

¿Afectará el método de pago?

¿Afectará el tipo de internet?

¿Afectará la edad?

¿El tiempo en la compañía es factor clave para no abandonarla?

In [ ]:
# Porcentaje de clientes que han abandonado
churn_rate = df['Churn'].mean() * 100
print(f"Tasa de abandono: {churn_rate:.2f}%")

In [ ]:
# Gráfico de barras de 'Churn'
plt.figure(figsize=(6,4))
sns.countplot(x='Churn', data=df, palette='Set2')
plt.title('Distribución de Churn')
plt.xlabel('Churn')
plt.ylabel('Número de Clientes')
plt.xticks([0,1], ['No', 'Sí'])
plt.show()


In [ ]:
# Gráfico de barras de 'Churn' por 'Contract'
plt.figure(figsize=(8,6))
sns.countplot(x='Contract', hue='Churn', data=df, palette='Set1')
plt.title('Churn por Tipo de Contrato')
plt.xlabel('Tipo de Contrato')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['No', 'Sí'])
plt.xticks(rotation=45)
plt.show()


Los clientes con contratos mes a mes tienen una tasa de abandono mucho más alta.

Los contratos a largo plazo están asociados con menor churn.

In [ ]:
# Gráfico de barras de 'Churn' por 'PaymentMethod'
plt.figure(figsize=(10,6))
sns.countplot(x='PaymentMethod', hue='Churn', data=df, palette='Set3')
plt.title('Churn por Método de Pago')
plt.xlabel('Método de Pago')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['No', 'Sí'])
plt.xticks(rotation=45)
plt.show()


El método de pago con cheque electrónico tiene una tasa de abandono más alta.
    
Los métodos automáticos como transferencia bancaria y tarjeta de crédito tienen tasas de abandono más bajas.

In [ ]:
# Gráfico de barras de 'Churn' por 'InternetService'
plt.figure(figsize=(8,6))
sns.countplot(x='InternetService', hue='Churn', data=df, palette='coolwarm')
plt.title('Churn por Tipo de Servicio de Internet')
plt.xlabel('Tipo de Servicio de Internet')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['No', 'Sí'])
plt.show()


Los clientes con servicio de fibra óptica tienen una tasa de abandono más alta.

Los clientes sin servicio de internet tienen la tasa de abandono más baja.

In [ ]:
# Gráfico de barras de 'Churn' por 'SeniorCitizen'
plt.figure(figsize=(6,4))
sns.countplot(x='SeniorCitizen', hue='Churn', data=df, palette='viridis')
plt.title('Churn por Senior Citizen')
plt.xlabel('Senior Citizen (0 = No, 1 = Sí)')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['No', 'Sí'])
plt.show()

Los clientes de la tercera edad tienen una **tasa de abandono** más alta.

Esto puede indicar la necesidad de adaptar servicios para este grupo demográfico.

In [ ]:
# Gráfico de densidad de 'tenure' por 'Churn'
plt.figure(figsize=(10,6))
sns.kdeplot(data=df, x='tenure', hue='Churn', fill=True)
plt.title('Distribución de Tenure por Churn')
plt.xlabel('Tenure (meses)')
plt.ylabel('Densidad')
plt.show()


Los clientes que han abandonado tienden a tener menor tiempo con la compañía.
    
La retención de clientes nuevos es un área clave a abordar.

# **7. Análisis de correlación**

Antes de calcular la matriz de correlación, debemos asegurarnos de que solo incluimos variables numéricas.

In [ ]:
# Seleccionar solo las columnas numéricas
numeric_df = df.select_dtypes(include=[np.number])

print("Columnas numéricas:", numeric_df.columns.tolist())

In [ ]:
# Matriz de correlación
corr_matrix = numeric_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(20,15))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()


Churn tiene una **correlación negativa** con tenure (-0.35), lo que indica que a **mayor** tiempo con la compañía, **menor** probabilidad de abandono.

MonthlyCharges tiene una **correlación positiva** con Churn (0.19), sugiriendo que **cargos mensuales más altos** están asociados con una **mayor tasa de abandono**.

## Conclusiones

* Clientes Nuevos: Los clientes con menor tiempo en la compañía (bajo tenure) son más propensos a abandonar. Es crucial enfocarse en estrategias de retención temprana.

* Contratos Mensuales: Los clientes con contratos mes a mes tienen tasas de abandono significativamente más altas. Incentivar contratos a largo plazo podría mejorar la retención.

* Métodos de Pago: Los clientes que pagan con cheque electrónico tienen mayor probabilidad de abandonar. Promover métodos de pago automáticos puede ser beneficioso.
